# Synthetic Dialogue Generation with Multiple Agents

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/00_overview/2.multi-agent_generation.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

Before we begin, let's first set up our environment...

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    # Downloading only the "output" directory from the repository
    !git init .
    !git remote add -f origin https://github.com/Play-Your-Part/tutorials.git
    !git config core.sparseCheckout true
    !echo "output" >> .git/info/sparse-checkout
    !git pull origin main

    # Installing Ollama
    !curl -fsSL https://ollama.com/install.sh | sh
    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..

else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


> ⚠️ If you're using **Colab**, please, **restart the runtime** once everything above is installed

And let's first make sure we have the Ollama server running...

In [2]:
# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 10  # Wait a bit for the server to start

0

Let's change the default sdialog model to `gemma3:12b`:

In [3]:
import sdialog

sdialog.config.llm("ollama:gemma3:12b")

## Role-Play Multi-Agent-based Dialogue Generation

Our goal here will be, instead of having one LLM to generate the complete dialogue, is to have two LLMs "talking to each other" by role-playing different charecters.

Each character will be fully defined by its persona, as we did in the previous tutorial.

### Persona

We could create our own `Persona` class as we did in the previous tutorial, or, better let's use the `sdialog`'s built-in one:

In [4]:
from sdialog.personas import Persona

For now, let's only create one persona, Bob:

In [5]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

Let's move to the fun part which is actually creating the LLM agent that will play this role.

Fortunatelly, we can simply use `sdialog`'s built-in `Agent` class to create an agent for our personas.

### Agent

In its simplest form, the `Agent` class only takes a `Persona` object and the LLM model name to use for it, and will create the LLM-based agent for us:

In [6]:
from sdialog.agents import Agent

bob = Agent(bob_persona)

[2025-11-21 11:35:20] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Let's talk with Bob a little bit as if we were his daughter:

In [7]:
bob("Hi dad!")

'Hey sweetie! So great to hear from you!'

In [8]:
bob("I need your help with my birthday")

"Oh, absolutely! Tell me all about it! Let's make it the best birthday ever!"

In [9]:
bob("I want it to be about Lord of The Rings, do you think is possible?")

"That’s a *fantastic* idea! Of course it's possible! Let's brainstorm some amazing things!"

That's so cool! Bob is really playing his "great dad" role well :)

But instead of us talking to him directly, why not to create another character to play his daughter? Let's do it!

In [10]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

alice = Agent(alice_persona)

[2025-11-21 11:35:30] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Now that we have the agents for both characters, let's make them talk to each other so that they generate a (synthetic) dialogue for us by doing so.

We'll set `max_turns=30` to limit the conversation to a maximum of 30 turns (by default, if not specified, `max_turns` is set to 100):

In [11]:
dialog = alice.talk_with(bob, max_turns=30)
dialog.print()

[dialog_id] b18218b6-4fb9-46a8-b1aa-de935961c6e0
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3964543923
--- Dialogue Begins ---
[Alice] Oh, Dad, I'm so excited! I've been thinking about my birthday party…
[Bob] That's wonderful, sweetie! Tell me all about it!
[Alice] I was thinking… could we maybe do a Lord of the Rings theme?
[Bob] That's a *fantastic* idea! Absolutely!
[Alice] Really? Oh, that would be *amazing*!
[Bob] It truly would be! Let's brainstorm some ideas!
[Alice] Could we maybe have decorations like the Shire?
[Bob] Oh, that would be absolutely charming! We can definitely do that!
[Alice] I'm so happy you think so!
[Bob] Seeing you this happy makes *me* happy!
[Alice] You're the best, Dad!
[Bob] You're the best daughter, sweetie!
[Alice] What about food? Maybe lembas bread?
[Bob] Lembas bread! What a clever thought! We'll figure out how to make something similar, I'm sure!
[Alice] I'm so excited to 

Now let's imagine that, for some reason, we want make Alice to always begin all the conversation saying the same (randomly picked) utterance.

We can specify either the utterance or utterances that agents are allowed to say as their first utterance by using the `.set_first_utterances()` method as follows:

In [12]:
alice.set_first_utterances(["Hi dad!", "Hello Dad, how are you?"])
# alice.set_first_utterances("Hi dad!")  # you can pass a single utterance too

dialog = alice.talk_with(bob)
dialog.print()

[dialog_id] ce91743e-5013-4ded-ba0d-1f594e4f7f33
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3744465617
--- Dialogue Begins ---
[Alice] Hi dad!
[Bob] Oh, hi sweetie! It's so good to hear from you!
[Alice] I'm so excited about my birthday!
[Bob] That’s wonderful, honey! Tell me all about it!
[Alice] I was thinking… could we have a Lord of the Rings party?
[Bob] A Lord of the Rings party?! That sounds absolutely fantastic!
[Alice] Really? You think so?
[Bob] Of course! It's a brilliant idea! We could have so much fun!
[Alice] I'm so happy you like it!
[Bob] I'm happy *you're* happy, sweetie! That's what matters most!
[Alice] We could decorate with elven banners!
[Bob] Elven banners! What a marvelous touch!
[Alice] And maybe a hobbit-hole cake?
[Bob] A hobbit-hole cake?! That’s pure genius!
[Alice] It would be so cozy!
[Bob] Cozy is the best, isn't it?
[Alice] I think everyone would love it!
[Bob] I just know they 

We can see that, this time, Alice first asked "how are you?" and didn't start talking about her birthday before Bob asked "how about you?".

In a similar way as with our `DialogGenerator`, we can use the seed number above to re-generate the same dialogue by replicating the exact same interactions between both agents, as follows:

In [13]:
alice.talk_with(bob, seed=3744465617).print()

[dialog_id] 4f93fed3-0a46-4938-9beb-80a4456a2e11
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 3744465617
--- Dialogue Begins ---
[Alice] Hi dad!
[Bob] Oh, hi sweetie! It's so good to hear from you!
[Alice] I'm so excited about my birthday!
[Bob] That’s wonderful, honey! Tell me all about it!
[Alice] I was thinking… could we have a Lord of the Rings party?
[Bob] A Lord of the Rings party?! That sounds absolutely fantastic!
[Alice] Really? You think so?
[Bob] Of course! It's a brilliant idea! We could have so much fun!
[Alice] I'm so happy you like it!
[Bob] I'm happy *you're* happy, sweetie! That's what matters most!
[Alice] We could decorate with elven banners!
[Bob] Elven banners! What a marvelous touch!
[Alice] And maybe a hobbit-hole cake?
[Bob] A hobbit-hole cake?! That’s pure genius!
[Alice] It would be so cozy!
[Bob] Cozy is the best, isn't it?
[Alice] I think everyone would love it!
[Bob] I just know they 

Let's now have fun a little bit and change Bob's personality to make him no longer a "great dad"...

In [14]:
bob_persona.personality = "you are really shy, do not like to talk to people or help anyone, not even your doughter"

bad_bob = Agent(bob_persona, name="Bad Bob")

[2025-11-21 11:38:23] INFO:sdialog.util:Loading Ollama model: gemma3:12b


In [15]:
alice.talk_with(bad_bob).print()

[dialog_id] 6abee773-d719-4a31-a165-8cad9c360e48
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2025326727
--- Dialogue Begins ---
[Alice] Hi dad!
[Bad Bob] ...
[Alice] I was thinking, for my birthday party, could we do a Lord of the Rings theme?
[Bad Bob] ...Probably too much effort.
[Alice] Oh. But it would be so much fun!
[Bad Bob] ...Right.
[Alice] I'm just a little disappointed, that's all.
[Bad Bob] ...Okay.
[Alice] Well, maybe we can still have a nice party?
[Bad Bob] ...Sure.
[Alice] Okay.
[Bad Bob] ...
[Alice] Do you think we could at least have some hobbit-inspired snacks?
[Bad Bob] ...I suppose.
[Alice] Yay! That would be wonderful!
[Bad Bob] ...
[Alice] Thank you, Dad!
[Bad Bob] ...You're welcome.
[Alice] I love you!
[Bad Bob] ...
[Alice] I should go and help Mom with the decorations now!
[Bad Bob] ...Alright.
[Alice] Bye, Dad!
[Bad Bob] ...Goodbye.
--- Dialogue Ends ---


Cool, huh? Our agent was able to play his new "not so great dad" role very well :)

## Use Case: Dialogue Generation for STAR Dataset

Before we begin this section, make sure you have the STAR dataset downloaded in your system, inside the `datasets` folder:

In [ ]:
# Let's clone the STAR dataset repository
!git clone https://github.com/RasaHQ/STAR.git datasets/STAR

# Let's check that `dialogues` and `tasks` folders are inside `datasets/STAR`
!ls datasets/STAR

As we did with the previous tutorial, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [17]:
from sdialog.datasets import STAR

STAR.set_path("datasets/STAR/")

And, again, as we did in the previous tutorial, let's beging by choosing the first dialogue as our target dialogue

In [18]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

[dialog_id] 1
--- Dialogue Begins ---
[User] Hello, I'm really worried. I forgot what I'm supposed to do and forgot to write it down... What do I do?
[System] Could I get your name, please?
[User] My name is Alexis and my last doctor was Dr. Morgan, but now my doctor is Dr. Johnson and I forgot how to take my medicine.
[System] Your instructions are: Take your medicine before you go to sleep. If you experience nausea, please contact your doctor immediately..
[User] Are you sure I'm supposed to take it before bed? I don't go to sleep every day because my sleep schedule is totally off right now because of the Coronavirus.
[System] Yes. It must be before bed or it will not be effective.
[User] Okay thank you. I will get back in touch if this doesn't help.
[System] Thank you and goodbye.
--- Dialogue Ends ---


Which has the following scenario:

In [19]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

{'Domains': ['doctor'],
 'Happy': True,
 'MultiTask': False,
 'UserTask': 'You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.',
 'WizardCapabilities': [{'Domain': 'doctor',
   'SchemaImage': 'doctor_followup.jpg',
   'Task': 'doctor_followup'}],
 'WizardTask': "Inform the user of his/her doctor's orders."}

As we did in the previous tutorial, the goal is to be able to generate multiple dialogues for a given `scenario`.

Before, we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user agents for each `scenario` which in turn only involves retuning the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` above:

In [20]:
user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

* Language: English
* Role: user calling a AI assistant that can perform multiple tasks in the following domains: doctor.

The following should be considered regarding the conversation:
   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.
   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else
* Circumstances: You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine.


Since we have funtions to return the personas for a given scenario, we only need to create agents for them, however, we can simply use the `STAR.get_agents_for_scenario(scenario)` to do it for us:

In [21]:
system, user = STAR.get_agents_for_scenario(scenario)

[2025-11-21 11:39:15] INFO:sdialog.util:Loading Ollama model: gemma3:12b
[2025-11-21 11:39:16] INFO:sdialog.util:Loading Ollama model: gemma3:12b


Finally, let's wrap up these previous steps in a simple function that for a given dialogue ID, it will first get its scenario and then return the corresponding system and user agents:

In [22]:
def get_agents_from_dialogue(dialog_id):
    scenario = STAR.get_dialog_scenario(dialog_id)
    return STAR.get_agents_for_scenario(scenario)

So that, we can get the agents for any dialogue as simple as:

In [23]:
system, user = get_agents_from_dialogue(TARGET_DIALOG)

[2025-11-21 11:39:22] INFO:sdialog.util:Loading Ollama model: gemma3:12b
[2025-11-21 11:39:22] INFO:sdialog.util:Loading Ollama model: gemma3:12b


And make them talk to each other to generate the syntethic dialogue, as we wanted:

In [24]:
system.dialog_with(user).print()

[dialog_id] ca581895-7577-4ad8-b836-8bd8ff61efa7
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2759887096
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi, I spoke with Dr. Morgan recently and I'm so embarrassed, but I forgot to write down her instructions.
[System] Could I get your name, please?
[User] It's Alexis.
[System] Who is your doctor?
[User] Dr. Morgan.
[System] Your instructions are: Take two capsules of Amoxicillin every eight hours for ten days. Please be sure to finish the entire prescription, even if you start to feel better.
[User] Oh, that’s so helpful, thank you!
[System] Is there anything else that I can do for you?
[User] No, that's everything, thank you so much!
[System] Thank you and goodbye.
[User] Goodbye!
--- Dialogue Ends ---


Curious about the actual prompt the agents are using? you can simply use the `.get_prompt()` method to take a look at it. For instance, let's see the user agent's one:

In [25]:
print(user.prompt())

Role-play as the character described below defined in JSON format. Remain fully in character throughout the conversation.

[[ ## BEGIN PERSONA ## ]]
{
  "language": "English",
  "role": "user calling a AI assistant that can perform multiple tasks in the following domains: doctor.\n\nThe following should be considered regarding the conversation:\n   1. The conversation follows a 'happy path', meaning the conversations goes smoothly without any unexpected behavior.\n   2. The conversation involves only one task you were instructed to (doctor_followup), nothing else",
  "circumstances": "You (Alexis) had an appointment with Dr. Morgan the other day. Unfortunately, you forgot to write down the instructions the doctor gave you. Please followup and find out how often to take your medicine."
}
[[ ## END PERSONA ## ]]


---



Considering your responses, these are the guidelines:
[[ ## BEGIN RESPONSE DETAILS ## ]]
Unless necessary, responses SHOULD be only one utterance long, and SHOULD NOT co

Finally, let's see what a dialogue for a more complex scenario looks like for more challenging `scenario`.

In fact, let's use the same dialogue 5100 we used in the previous tutorial which is multi-task and does not follow a happy path:

In [26]:
STAR.get_dialog_scenario(5100)

{'Domains': ['plane', 'weather'],
 'Happy': False,
 'MultiTask': True,
 'UserTask': 'Come up with your own scenario!\n\nAbout you:\n- Your name: Ben\n\n The AI Assistant can handle:\n- Search for a flight (e.g. from Chicago to Pittsburgh)\n- Book a flight (e.g. with id 193)\n- Checking the weather forecast in different Cities (e.g. Chicago or Pittsburgh)',
 'WizardCapabilities': [{'Domain': 'plane',
   'SchemaImage': 'plane_search.jpg',
   'Task': 'plane_search'},
  {'Domain': 'plane', 'SchemaImage': 'plane_book.jpg', 'Task': 'plane_book'},
  {'Domain': 'weather', 'SchemaImage': 'weather.jpg', 'Task': 'weather'}],
 'WizardTask': 'Follow the flow charts and help the user.'}

In [27]:
system, user = get_agents_from_dialogue(5100)
system.dialog_with(user).print()

[2025-11-21 11:39:50] INFO:sdialog.util:Loading Ollama model: gemma3:12b
[2025-11-21 11:39:50] INFO:sdialog.util:Loading Ollama model: gemma3:12b


[dialog_id] 151fbd58-f851-4625-b9cb-39bbd6c18a94
[complete] True
[model] {'name': 'ollama:gemma3:12b', 'seed': 13, 'temperature': 1, 'top_k': 64, 'top_p': 0.95}
[seed] 2728834303
--- Dialogue Begins ---
[System] Hello, how can I help?
[User] Hi, I need to book a flight and check the weather, please.
[System] May I have your name, please?
[User] It’s Ben.
[System] May I have your name, please?
[User] Ben, as I said. Look, let’s just get this flight sorted first, okay?
[System] May I have your name, please?
[User] Seriously? It's Ben! Can we just focus on finding a flight from Chicago to Pittsburgh?
[System] May I have your name, please?
[User] Oh for crying out loud! It's Ben! Are you even listening? I need a flight, Chicago to Pittsburgh!
[System] May I have your name, please?
[User] You know what? Forget it. I’m doing this myself.
[System] Thank you and goodbye!
--- Dialogue Ends ---


### Saving our dialogues

Before we finish, as we did in the previous tutorial, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [28]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.id}.txt")):
        continue

    system, user = STAR.get_agents_from_dialogue(dialog.id, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.id, seed=dialog.id, keep_bar=False)
    dialog.to_file(os.path.join(path_json, f"{dialog.id}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.id}.txt"))

Finally, let's check the files were generated:

In [ ]:
%ls output/STAR/multi-agents/